In [1]:
import os
import sys
import random
import json
import collections

import pandas as pd
import numpy as np
import scipy
import statsmodels
from tqdm import trange, tqdm_notebook as tqdm

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline


pd.options.display.max_columns = 999

sns.set()
plt.rcParams["figure.figsize"] = (10,7)

In [2]:
os.getcwd()

'/notebooks/anton/sigmod/notebooks'

In [105]:
LG_LABELS_PATH = '../data/raw/sigmod_large_labelled_dataset.csv'

In [109]:
lg_labels_df = pd.read_csv(LG_LABELS_PATH)
lg_labels_df.head()

,left_spec_id,right_spec_id,label
0,www.ebay.com//53278,www.garricks.com.au//31,1
1,www.ebay.com//24817,www.ebay.com//58782,0
2,www.ebay.com//43019,www.ebay.com//58782,0
3,www.ebay.com//42055,www.ebay.com//54403,0
4,buy.net//6145,www.ebay.com//44280,0


### Get Brands for all products

In [151]:
from tqdm import tqdm_notebook

In [19]:
SPECS_PATH = '../data/raw/2013_camera_specs/'

In [739]:
def read_json(path):
    with open(path, 'r') as f:
        return json.load(f)
    
def pprint_json(json_):
    print(json.dumps(json_, indent=4, sort_keys=True))

getting all titles and brands form specs

In [924]:
from collections import defaultdict
brands = defaultdict(list)
titles = []
sps = []
title_name = []
for site in site_folders:
    specs = os.path.join(SPECS_PATH, site)
    for i, spec in enumerate(os.listdir(specs)):
        spec_path = os.path.join(specs, spec)
        js = read_json(spec_path)
        brand = js.get('brand')
        title = js.get('<page title>')
        if isinstance(brand, list):
            continue
        if brand:
            brand = re.sub('[^a-z0-9] ', '', brand.lower())
            brands[brand].append(len(titles))
        title = re.sub('[^a-z0-9]', ' ', title.lower())
        title = re.sub(' +', ' ', title)
        titles.append(title)
        title_name.append(site+'//'+spec[:-5])
        sps.append(js)

add some brands from internal knowledges

In [864]:
list_of_brands = list(brands.keys())
list_of_brands.append('cctv')
list_of_brands.append('ip')
list_of_brands.append('binoculars')
list_of_brands.append('sekonic')
list_of_brands.append('samyang')
list_of_brands.append('lexar')
list_of_brands.append('ksm')
list_of_brands.append('uv')
list_of_brands.append('hoya')
list_of_brands.append('dahua')
list_of_brands.append('hikvision')
list_of_brands.append('colorpix')
list_of_brands.append('onvif')

drop all brands with less then 10 examples

In [865]:
br_in = defaultdict(int)
br_num = defaultdict(list)
emp = []
for i, title in tqdm_notebook(enumerate(titles)):
    list_title = title.split(' ')
    z = 0
    for word in list_title:
        if word in list_of_brands:
            br_in[word] += 1
            br_num[word].append(i)
            z = 1
    if z == 0:
        emp.append(i)
        
for key, val in tuple(br_in.items()):
    if val < 10:
        br_in.pop(key)
        br_num.pop(key)
        emp.append(k)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


for each title get it's brand number

In [893]:
br_num['nonbrand'] = 0
not_brand = set(('eos', 'action', 'shoot', 'new', 'class', 'neopine', 'ion', 'sharp', 'fuji', 'tamron'))
brands = set(br_num.keys()) - not_brand

In [894]:
brand_to_num = {brand:i for i, brand in enumerate(brands)} 
num_to_brand = {i:brand for i, brand in enumerate(brands)} 

In [895]:
br_finish = defaultdict(list)
title_labels = []
for i, title in tqdm_notebook(enumerate(titles)):
    title = re.sub('[^a-z0-9]', ' ', title.lower())
    title = re.sub(' +', ' ', title)
    title = title.replace('cannon', 'canon')

    if 'fuji' in title:
        title_labels.append(brand_to_num['fujifilm'])
        br_finish['fujifilm'].append(i)
        continue

    if 'eos' in title:
        title_labels.append(brand_to_num['canon'])
        br_finish['canon'].append(i)
        continue

    for br in brands:
        if br + ' ' in title or ' ' + br in title:
            title_labels.append(brand_to_num[br])
            br_finish[br].append(i)
            break
    else:
        title_labels.append(brand_to_num['nonbrand'])
        br_finish['nonbrand'].append(i)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


len(set(tuple(br_num['telesin'])) & set(tuple(br_num['gopro']))) # not shure merge or not

len(set(tuple(br_num['dv'])) & set(tuple(br_num['sony'])))

Check whether are dublicates from labels in the came class 

In [872]:
LG_LABELS_PATH = '../data/raw/sigmod_large_labelled_dataset.csv'
labels_df = pd.read_csv(LG_LABELS_PATH)
title_name = np.array(title_name)

In [923]:

bad_examples = []
for i, (first_it, second_it, cl) in tqdm_notebook(enumerate(np.array(labels_df))):
    id_f = np.where(title_name == first_it)[0][0]
    id_sec = np.where(title_name == second_it)[0][0]

    if title_labels[id_f] != title_labels[id_sec] and cl == '1':
        print('ALERT')
        bad_examples.append([first_it, second_it, cl, id_f, id_sec])
#     elif title_labels[id_f] == title_labels[id_sec] and:
#         print('ALERT2')
#         break

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


### Get model for all products

In [931]:
from collections import defaultdict
models = defaultdict(list)
z = 0
for site in site_folders:
    specs = os.path.join(SPECS_PATH, site)
    for i, spec in enumerate(os.listdir(specs)):
        spec_path = os.path.join(specs, spec)
        js = read_json(spec_path)
        model = js.get('model')
        if isinstance(model, list):
            model = ' '.join(model)
        if model:
            model = re.sub('[^a-z0-9] ', '', model.lower())
            models[model].append(z)
        z +=1

In [941]:
from sigmod_src.data.make_dataset import make_specs_dataset, preprocess_specs_dataset, make_labelled_dataset 
from sigmod_src.features.build_features import make_features

In [938]:
SPECS_PATH = '../data/raw/2013_camera_specs/'
specs_df = make_specs_dataset(SPECS_PATH)
specs_df = preprocess_specs_dataset(specs_df)

Warning, dropped 16 rows containing null page titles


NameError: name 'make_features' is not defined

In [942]:
spec_features_df, vectorizers, site_le = make_features(specs_df)
spec_features_df = specs_df.merge(spec_features_df, on='spec_id', how='left')

In [943]:
labels_df = make_labelled_dataset(LG_LABELS_PATH, spec_features_df)

In [ ]:
labels_df.to_csv('../data/processed/labelled_features.csv', index=None)

In [ ]:
labelled_dataset_path = '../data/processed/labelled_features.csv'
df = pd.read_csv(labelled_dataset_path)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop(['label', 'left_spec_id', 'right_spec_id'], axis=1)
Y = df.label

In [ ]:
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, stratify=Y)
train_Y.shape, test_Y.shape

In [24]:
from sklearn.metrics import f1_score, plot_confusion_matrix, classification_report

In [23]:
from lightgbm import LGBMClassifier

In [25]:
train_Y.value_counts()

0    32312
1     2686
Name: label, dtype: int64

In [26]:
test_Y.value_counts()

0    10771
1      896
Name: label, dtype: int64

In [28]:
sample_pos_weight = Y.value_counts()[0] / Y.value_counts()[1]
sample_pos_weight

12.027638190954773

In [29]:
clf = LGBMClassifier(sample_pos_weight=sample_pos_weight)

In [30]:
clf.fit(train_X, train_Y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               learning_rate=0.1, max_depth=-1, min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
               n_jobs=-1, num_leaves=31, objective=None, random_state=None,
               reg_alpha=0.0, reg_lambda=0.0,
               sample_pos_weight=12.027638190954773, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=1)

In [37]:
pred_train_Y = clf.predict(train_X)

In [38]:
print('Train F1', f1_score(train_Y, pred_train_Y))
print(classification_report(train_Y, pred_train_Y))

Train F1 0.9992548435171386
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     32312
           1       1.00      1.00      1.00      2686

    accuracy                           1.00     34998
   macro avg       1.00      1.00      1.00     34998
weighted avg       1.00      1.00      1.00     34998

